In [1]:
import pandas as pd
import random
random.seed(2)

which_wiki = 1

# Read in the data
df = pd.read_csv(f'all-outputs-new-wiki-{which_wiki}.csv')

In [ ]:
import json

annotater_dict = json.load(open('annotater_dict-new-wiki-1.json'))
id_dict = json.load(open('id_dict-new-wiki-1.json'))

# print length of each annotater's list
for annotater in annotater_dict:
    print(annotater, len(annotater_dict[annotater]))

In [19]:
annotater_dict

{'vinayak': [0, 1, 3, 7, 8, 10, 11, 13, 16, 18, 19, 20, 21, 22, 23],
 'kelly': [0, 3, 4, 5, 9, 12, 13, 14, 15, 17, 23, 24, 25, 26, 28],
 'rachel': [1, 2, 10, 11, 14, 16, 19, 21, 23, 24, 25, 26, 27, 28, 29],
 'vishnesh': [1, 3, 4, 5, 6, 7, 9, 11, 14, 15, 16, 17, 18, 20, 29],
 'ayush': [2, 5, 6, 7, 8, 9, 12, 13, 17, 18, 21, 22, 26, 27, 28],
 'anton': [0, 2, 4, 6, 8, 10, 12, 15, 19, 20, 22, 24, 25, 27, 29]}

In [20]:
import re

system_list = ["Human 1 Writing", "Human 2 Writing", "GPT-3-zero-shot", "GPT-3-few-shot", "Muss"]
for annotater in annotater_dict:
    annotater_batch = []
    start_id = 0
    for id in annotater_dict[annotater]:
        # get the id th row of df
        row = df.iloc[id]
        random.seed(2)
        # random shuffle the system list
        random.shuffle(system_list)
        for system in system_list:
            current_dict = {}
            current_dict["id"] = start_id
            current_dict["original"] = row["Original Sentence"].strip()
            current_dict["original_spans"] = []

            simplified = row[system].strip()
            simplified = re.sub(r" 's", "'s", simplified)
            current_dict["simplified"] = simplified
            current_dict["simplified_spans"] = []

            if "||" in simplified:
                #  find index of all || in simplified
                indices = [m.start() for m in re.finditer('\|\|', simplified)]
                for i, indice in enumerate(indices):
                    current_dict["simplified_spans"].append([2, indice, indice+2, i])
            
            current_dict["system"] = "new-wiki-1/" + system
            annotater_batch.append(current_dict)
            start_id += 1

    # split annotater_batch into 2 batches first 40 and last 35
    annotater_batch_1 = annotater_batch[:40]
    annotater_batch_2 = annotater_batch[40:]

    with open("../batches/new-wiki-1/part1/" + annotater + ".json", "w") as f:
        json.dump(annotater_batch_1, f, indent=4)

    # reindex annotater_batch_2's id from 0
    for i, item in enumerate(annotater_batch_2):
        item["id"] = i

    with open("../batches/new-wiki-1/part2/" + annotater + ".json", "w") as f:
        json.dump(annotater_batch_2, f, indent=4)